# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
weather_csv = "weather.csv"
weather = pd.read_csv(weather_csv)

weather.head()

,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,port lincoln,0,1592425141,63,-34.73,135.87,55.92,19.33
1,new norfolk,0,1592425141,88,-42.78,147.06,33.01,2.91
2,bathsheba,40,1592425141,66,13.22,-59.52,86.00,19.46
3,katsuura,75,1592425141,88,35.13,140.30,69.80,8.05
4,qaanaaq,45,1592424908,89,77.48,-69.36,34.09,3.31


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Set locations and humidity values 
locations = weather[["Lat", "Lng"]]
humidity = weather["Humidity"]

humidity.head()

0    63
1    88
2    66
3    88
4    89
Name: Humidity, dtype: int64

In [19]:
# Build Humidity Heatmap
figure = {
    'width': '900px',
    'height': '600px',
    'border': '1px black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

heatmap = gmaps.figure(layout=figure,zoom_level=2,center=(15,25))

#Set overlay properties
heat_overlay = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=80,
                                 point_radius=1.5)

heatmap.add_layer(heat_overlay)

heatmap

Figure(layout=FigureLayout(border='1px black', height='600px', margin='0 auto 0 auto', padding='1px', width='9…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
# Set the ideal weather conditions.
temp = (weather["Max Temp"] < 80) & (weather["Max Temp"] > 70)
wind = weather["Windspeed"] < 10
clouds = weather["Cloudiness"] == 0
conditions = temp & wind & clouds 

# Put ideal locations into a new dataframe
ideal_weather = weather[conditions]
ideal_weather = ideal_weather.reset_index()
ideal_weather.head(50)

,index,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,36,corinto,0,1592425146,53,37.94,22.96,73.00,5.99
1,114,erzin,0,1592425219,84,36.96,36.20,75.00,0.25
2,191,namibe,0,1592425351,70,-15.20,12.15,70.72,3.74
3,208,srednekolymsk,0,1592425354,34,67.45,153.68,78.12,7.81
4,210,burns lake,0,1592425354,30,54.23,-125.75,71.71,2.91
5,238,little current,0,1592425419,45,45.98,-81.92,75.99,1.99
6,263,benguela,0,1592425423,78,-12.58,13.41,70.99,3.13
7,305,eureka,0,1592425490,26,40.80,-124.16,73.00,8.01
8,338,ashqelon,0,1592425494,42,31.67,34.57,78.01,7.67
9,372,qui nhon,0,1592425560,75,13.77,109.23,78.91,4.97


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
hotel_df = []

# Ideal Weather dataframe to find closest hotels
for city in range(len(ideal_weather["City"])):

    lat = ideal_weather.loc[city]["Lat"]
    lng = ideal_weather.loc[city]["Lng"]

    coordinates = f"{lat},{lng}"

    params = {
        "location": coordinates, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    request = requests.get(base_url, params=params)
    response = request.json()

    try:
        hotel_df.append(response["results"][0]["name"])
    except:
        hotel_df.append("Nearest hotel not found")

# New dataframe with closest hotel names
ideal_weather["Hotel Name"] = hotel_df
ideal_weather

,index,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Windspeed,Hotel Name
0,36,corinto,0,1592425146,53,37.94,22.96,73.00,5.99,Ephira Hotel
1,114,erzin,0,1592425219,84,36.96,36.20,75.00,0.25,Artemis Otel
2,191,namibe,0,1592425351,70,-15.20,12.15,70.72,3.74,Hotel Chik Chik Namibe
3,208,srednekolymsk,0,1592425354,34,67.45,153.68,78.12,7.81,Nearest hotel not found
4,210,burns lake,0,1592425354,30,54.23,-125.75,71.71,2.91,Burns Lake Bed and Breakfast
5,238,little current,0,1592425419,45,45.98,-81.92,75.99,1.99,Hawberry Motel
6,263,benguela,0,1592425423,78,-12.58,13.41,70.99,3.13,Hotel Praia Morena
7,305,eureka,0,1592425490,26,40.80,-124.16,73.00,8.01,Eureka Inn Trademark By Wyndham
8,338,ashqelon,0,1592425494,42,31.67,34.57,78.01,7.67,Leonardo Ashkelon
9,372,qui nhon,0,1592425560,75,13.77,109.23,78.91,4.97,Khách sạn Hải Âu Quy Nhơn - Seagull Hotel


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]
locations = ideal_weather[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map